In [8]:
from rec_generator import *
from rec_gen_utils.solver import *
from multiprocessing import Pool
from rec_gen_utils.thread_help import mute

class MakeMeReadable:
    def __init__(self, d):
        self.d = d
    
    def __dir__(self):
        return self.d.keys()
    
    def __getattr__(self, v):
        try:
            out = self.d[v]
            if isinstance(out, dict):
                return MakeMeReadable(out)
            return out
        except:
            return getattr(self.d, v)
    
    def __getitem__(self, v):
        return self.__getattr__(v)
        
    def __str__(self):
        print("wat")
        return str(self.d)
    
    def __repr__(self):
        return repr(self.d)

In [9]:
import pandas as pd
import json
import os
root_dir = os.path.expanduser('~') + "/OneDrive/wp1/"
#######
# Basic limits
#######

battery_cost_y1 = 518.64
battery_cost_y15 = 318.14

#######
# Demand/injection
#######
agg_demand = pd.read_parquet(root_dir + "data/full_mrs/meterings.parquet")

demand = agg_demand.clip(0, None)
injection = -agg_demand.clip(None, 0)

#######
# PV limits per company
#######
pv_capacity_bounds = json.load(open(root_dir + 'data/pv_limits_per_company.json'))
for name in demand.columns:
    if name not in pv_capacity_bounds:
        pv_capacity_bounds[name] = [0.0, 0.0]

#######
# Compute various hourly costs
#######

grid_taxes = json.load(open(root_dir + "data/prices/additionnal.json"))
days_per_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
grid_import_price = pd.read_parquet(root_dir + "data/prices/import_prices.parquet")
grid_export_price = pd.read_parquet(root_dir + "data/prices/export_prices.parquet")
rec_surcharge = pd.read_parquet(root_dir + "data/prices/rec_surcharge.parquet")

peak_month_price = grid_taxes["peak_month"]
peak_month_price_deg = grid_taxes["peak_month_discount"]
peak_hist_price = grid_taxes["peak_hist"]
peak_hist_price_deg = grid_taxes["peak_hist_discount"]
peak_demand_1st_to_11th_coef = 1.0

members = [
    RECMember(
        name=name,
        # we set the minimum at 0 because the current PV is already counted in the AMRs
        pv_maximum_capacity=pv_capacity_bounds[name][1] - pv_capacity_bounds[name][0],
        pv_minimum_capacity=0,
        battery_maximum_capacity=3*pv_capacity_bounds[name][1],
        battery_minimum_capacity=0,
        battery_cost_y1=battery_cost_y1,
        battery_cost_y15=battery_cost_y15,
        demand=demand[name].to_numpy(),
        base_injection=injection[name].to_numpy(),
        grid_import_price=grid_import_price[name].to_numpy(),
        grid_export_price=grid_export_price[name].to_numpy(),
        rec_energy_surcharge=rec_surcharge[name].to_numpy(),
        peak_month_price=peak_month_price,
        peak_month_price_deg=peak_month_price_deg,
        peak_hist_price=peak_hist_price,
        peak_hist_price_deg=peak_hist_price_deg,
        peak_demand_1st_to_11th_coef=peak_demand_1st_to_11th_coef)
    for name in demand.columns
]

zoning_current = REC(members, allow_rec_exchanges=False, grid_environmental_impact=np.loadtxt(root_dir + "data/impact_grid_2019.csv"))
zoning_rec = REC(members, allow_rec_exchanges=True, grid_environmental_impact=np.loadtxt(root_dir + "data/impact_grid_2019.csv"))

In [10]:
def modify_member(orec, what, how):
    def intern(m):
        return dataclasses.replace(m, **{what: how(m)})
    return dataclasses.replace(orec, members=[intern(m) for m in orec.members])
def compute_env_impact(readable):
    return sum(getattr(readable.solution.elements, f"MEMBER_{i}").variables.environmental_impact.values[0] for i in range(28))

zoning_noinvest = modify_member(zoning_current, "pv_maximum_capacity", lambda x: 0)
zoning_noinvest = modify_member(zoning_current, "battery_maximum_capacity", lambda x: 0)
zoning_noinvest_rec = modify_member(zoning_rec, "pv_maximum_capacity", lambda x: 0)
zoning_noinvest_rec = modify_member(zoning_rec, "battery_maximum_capacity", lambda x: 0)

In [11]:

rec = zoning_current#gen_and_solve(zoning_current, Path(os.path.expanduser('~') + "/.tmp"), base_path=root_dir)
import dataclasses, json

class EnhancedJSONEncoder(json.JSONEncoder):
        def default(self, o):
            if dataclasses.is_dataclass(o):
                return dataclasses.asdict(o)
            elif isinstance(o, np.ndarray):
                return o.tolist()
            return super().default(o)
with open(os.path.expanduser('~') + "/rec_paper_peaks_code/envs/big_rec_data/big_rec.json", "w") as frec:
    json.dump(rec, frec, cls=EnhancedJSONEncoder)
#solution_rec = gen_instance(Path(os.path.expanduser('~') + "/.tmp"), zoning_rec, base_path=root_dir)#gen_and_solve(zoning_rec, Path(os.path.expanduser('~') + "/.tmp"), base_path=root_dir)
#print(zoning_current)
#print(zoning_rec)

In [12]:
def mul_pv_max(orec, x=1.0):
    return modify_member(orec, "pv_maximum_capacity", lambda m: m.pv_maximum_capacity * x)

with Pool(3, initializer=mute) as pool:
    def gen_and_solve_async(rec):
        return pool.apply_async(gen_and_solve, (rec,))
    sensi_pv = [(i, 
                 gen_and_solve_async(mul_pv_max(zoning_current, i)), 
                 gen_and_solve_async(mul_pv_max(zoning_rec, i))) for i in np.arange(0, 3, 0.25, dtype=float)]
    sensi_pv = [(i, MakeMeReadable(j.get()), MakeMeReadable(k.get())) for i, j, k in sensi_pv]
    sensi_pv_bat = [(i, 
                 *[getattr(j.solution.elements, f"MEMBER_{a}").sub_elements.BATTERY.variables.capacity.values[0] for a in range(28)],
                 *[getattr(k.solution.elements, f"MEMBER_{a}").sub_elements.BATTERY.variables.capacity.values[0] for a in range(28)],
                )
                for i, j, k in sensi_pv]
    sensi_pv_env = [(i, compute_env_impact(j), compute_env_impact(k)) for i, j, k in sensi_pv]
    sensi_pv = [(i, j.solution.objective, k.solution.objective) for i, j, k in sensi_pv]
sensi_pv

FileNotFoundError: [Errno 2] No such file or directory: 'tmp/6094130e737b5b218a6382eac9b475f8d5c8215799c66da84632db9533d122c9'

In [ ]:
pd.DataFrame(sensi_pv_bat, columns=['mul']+[f'cur_{i}' for i in range(28)]+[f'rec_{i}' for i in range(28)]).set_index("mul").sum(axis=1)

In [ ]:
pd.DataFrame(sensi_pv_env, columns=['mul', 'env_cur', 'env_rec']).set_index("mul")

In [ ]:
def mul_surcharge(orec, x=1.0):
    return modify_member(orec, "rec_energy_surcharge", lambda m: m.rec_energy_surcharge * x)

with Pool(3, initializer=mute) as pool:
    def gen_and_solve_async(rec):
        return pool.apply_async(gen_and_solve, (rec,))
    sensi_surcharge = [(i, gen_and_solve_async(mul_surcharge(zoning_rec, i))) for i in np.arange(0, 1.01, 0.1, dtype=float)]
    sensi_surcharge = [(i, MakeMeReadable(j.get())) for i, j in sensi_surcharge]
    sensi_surcharge_bat = [(i, *[getattr(j.solution.elements, f"MEMBER_{k}").sub_elements.BATTERY.variables.capacity.values[0] for k in range(28)]) for i, j in sensi_surcharge]
    sensi_surcharge = [(i, j.solution.objective) for i, j in sensi_surcharge]    
sensi_surcharge

In [ ]:
pd.DataFrame(sensi_surcharge_bat, columns=['mul']+[f'rec_{i}' for i in range(28)]).set_index("mul").sum(axis=1)

In [ ]:
def mul_battery_max(orec, x):
    return modify_member(orec, "battery_maximum_capacity", lambda m: m.pv_maximum_capacity * x)

with Pool(3, initializer=mute) as pool:
    def gen_and_solve_async(rec):
        return pool.apply_async(gen_and_solve, (rec,))
    sensi_bat = [(i, 
                 gen_and_solve_async(mul_battery_max(zoning_current, i)), 
                 gen_and_solve_async(mul_battery_max(zoning_rec, i))) for i in np.arange(0, 6.1, 0.5, dtype=float)]
    sensi_bat = [(i, j.get()['solution']['objective'], k.get()['solution']['objective']) for i, j, k in sensi_bat]
sensi_bat

In [ ]:
def mul_battery_max_p(orec, x):
    orec = modify_member(orec, "battery_cost_y1", lambda m: m.battery_cost_y1 * x)
    orec = modify_member(orec, "battery_cost_y15", lambda m: m.battery_cost_y15 * x)
    return orec

with Pool(3, initializer=mute) as pool:
    def gen_and_solve_async(rec):
        return pool.apply_async(gen_and_solve, (rec,))
    sensi_bat_p = [(i, 
                 gen_and_solve_async(mul_battery_max_p(zoning_current, i)), 
                 gen_and_solve_async(mul_battery_max_p(zoning_rec, i))) for i in np.arange(0.1, 1.21, 0.1, dtype=float)]
    
    
    sensi_bat_p = [(i, MakeMeReadable(j.get()), MakeMeReadable(k.get())) for i, j, k in sensi_bat_p]
    sensi_bat_p_bat = [(i, 
                 sum([getattr(j.solution.elements, f"MEMBER_{a}").sub_elements.BATTERY.variables.capacity.values[0] for a in range(28)]),
                 sum([getattr(k.solution.elements, f"MEMBER_{a}").sub_elements.BATTERY.variables.capacity.values[0] for a in range(28)]),
                )
                for i, j, k in sensi_bat_p]
    sensi_bat_p = [(i, j.solution.objective, k.solution.objective) for i, j, k in sensi_bat_p]
sensi_bat_p

In [ ]:
pd.DataFrame(sensi_bat_p_bat, columns=['mul', 'installed_cur', 'installed_rec']).set_index("mul")

In [ ]:
def modify(orec, x):
    orec = modify_member(orec, "grid_import_price", lambda m: (m.grid_import_price - m.rec_energy_surcharge)*x + m.rec_energy_surcharge)
    orec = modify_member(orec, "grid_export_price", lambda m: m.grid_export_price*x)
    return orec

with Pool(3, initializer=mute) as pool:
    def gen_and_solve_async(rec):
        return pool.apply_async(gen_and_solve, (rec,))
    sensi_elec_p = [(i, 
                     gen_and_solve_async(modify(zoning_current, i)), 
                     gen_and_solve_async(modify(zoning_rec, i)),
                     gen_and_solve_async(modify(zoning_noinvest, i)),
                     gen_and_solve_async(modify(zoning_noinvest_rec, i))
                    ) 
                    for i in np.arange(1, 4.1, 0.25, dtype=float)
                   ]
    sensi_elec_p = [(i, MakeMeReadable(j.get()), MakeMeReadable(k.get()), MakeMeReadable(l.get()), MakeMeReadable(m.get())) for i, j, k, l, m in sensi_elec_p]
    sensi_elec_p_bat = [(i, 
                         sum([getattr(j.solution.elements, f"MEMBER_{a}").sub_elements.BATTERY.variables.capacity.values[0] for a in range(28)]),
                         sum([getattr(k.solution.elements, f"MEMBER_{a}").sub_elements.BATTERY.variables.capacity.values[0] for a in range(28)]),
                         sum([getattr(l.solution.elements, f"MEMBER_{a}").sub_elements.BATTERY.variables.capacity.values[0] for a in range(28)]),
                         sum([getattr(m.solution.elements, f"MEMBER_{a}").sub_elements.BATTERY.variables.capacity.values[0] for a in range(28)]),
                        )
                        for i, j, k, l, m in sensi_elec_p]
    sensi_elec_p = [(i, 
                     j.solution.objective, 
                     k.solution.objective,
                     l.solution.objective,
                     m.solution.objective
                    ) for i, j, k, l, m in sensi_elec_p]
sensi_elec_p

In [ ]:
df = pd.DataFrame(sensi_elec_p_bat, columns=["Multiplicateur prix elec", "no_rec", "rec", "no_invest_no_rec", "no_invest_rec"]).set_index("Multiplicateur prix elec")
df[["no_rec", "rec"]].plot(figsize=(16/2,9/2), title="Capacité batterie installée en fonction du prix de l'électricité (kWh)")

In [ ]:
sum((d.members[i].grid_import_price.mean() - d.members[i].rec_energy_surcharge.mean())*1000 for i in range(28))/28

In [ ]:
df = pd.DataFrame(sensi_elec_p, columns=["Multiplicateur prix elec", "no_rec", "rec", "no_invest_no_rec", "no_invest_rec"]).set_index("Multiplicateur prix elec")
(100.0*(df.no_rec - df.rec)/(df.no_rec)).plot(figsize=(16/2,9/2), title="Economie supplémentaire en REC (%) en fonction du prix de l'électricité")

In [ ]:
df.plot(figsize=(16/2,9/2), title="Coûts totaux en fonction du prix de l'électricité (M€)")

In [ ]:
df.sub(df.rec, axis=0).plot(figsize=(16/2,9/2), title="Coûts supplémentaires hors-invest hors-rec (€)")

In [ ]:
df["diff"] = df.no_rec - df.rec
df["diff"].plot()

In [ ]:
pd.DataFrame(sensi_surcharge, co)

In [ ]:
d = MakeMeReadable(gen_and_solve(mul_battery_max(zoning_current, 1.0)))
gen_instance(Path("tmp"), mul_battery_max(zoning_current, 1.0))

In [ ]:
def mul_battery_max(orec, x, p):
    orec = modify_member(orec, "battery_maximum_capacity", lambda m: m.pv_maximum_capacity * x)
    orec = modify_member(orec, "battery_cost_y1", lambda m: p)
    orec = modify_member(orec, "battery_cost_y15", lambda m: p)
    return orec


a = gen_and_solve(mul_battery_max(zoning_current, 1.0, 10))['solution']['objective']
b = gen_and_solve(mul_battery_max(zoning_current, 1.0, 100))['solution']['objective']
c = gen_and_solve(mul_battery_max(zoning_current, 1.0, 300))['solution']['objective']
print(a, b, c)

In [ ]:
d = MakeMeReadable(solution_current)

In [ ]:
compute_env_impact(d)

In [ ]:
zoning_current.members[12]

In [ ]:
d.solution.elements.MEMBER_12.sub_elements.PV.variables.costs.values[0]/d.solution.elements.MEMBER_12.sub_elements.PV.variables.capacity.values[0]

In [ ]:
for i in range(27):
    costs = getattr(d.solution.elements, f"MEMBER_{i}").sub_elements.PV.variables.costs.values[0]
    capa = getattr(d.solution.elements, f"MEMBER_{i}").sub_elements.PV.variables.capacity.values[0]
    print(costs/capa/0.07 if capa != 0.0 else 0)

In [ ]:
for x in d.solution.elements.d:
    if "MEMBER_" in x:
        print(d.solution.elements[x].sub_elements.BATTERY.variables.capacity.values[0])

In [ ]:
(solution_rec['solution']['objective'] - solution_current['solution']['objective'])/solution_current['solution']['objective']

In [ ]:
solution_rec_invest = solution_rec
solution_current_invest = solution_current

In [ ]:
solution_rec['solution']['objective'] - solution_current['solution']['objective']

In [ ]:
demand['NRB'].plot(figsize=(15, 15)) #TODO: regarder ce qu'il se passe. wut.

In [ ]:
demand.plot(figsize=(15, 15))